In [2]:
# set sql connection
import pyodbc
server = 'RIC1D2RSGSQL01' 
database = 'EquityResearch' 
conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';Trusted_Connection=yes;')

In [3]:
import pandas as pd

### 1. One simple example
Here I will demonstrate how you can create a sql query and execute it from your Python IDE to retrieve the data into a dataframe.

In [18]:
# define your parameters for SQL query
# Note to use strip [] from a list variable, 
# It will give back str with comma delimitered & single quoted, exactly what SQL needed

# Dates in a str (feel free to change the query for dates selection, it can be daily between a & b, or in a list of dates)
# Security in a str


Dates = ['2019-01-31','2019-02-28','2019-03-29']
Security = ['AMZN','AAPL'] 
# In most of the case, you will assign a list of unique securities from an existing df
# make sure the variable type should be a LIST, e.g. using .tolist() 
# e.g. SecList = sma_sig['Ticker'].unique().tolist()

DateList = str(Dates).strip('[]') 
SecList = str(Security).strip('[]')

query = f'''SELECT SecurityId, TReturn, Date, Flag
--select count(1)
FROM EquityResearch.dbo.ReturnsHistory rh
	INNER JOIN securitydb.dbo.SecurityMaster_Hist sm
		ON rh.IntegerId = sm.IntegerId
		AND rh.Date BETWEEN sm.StartDate AND sm.EndDate
		-- AND LEFT(rh.IntegerId,1) = '1' --for US names
AND rh.Date IN ({DateList})
AND sm.Ticker IN ({SecList})
ORDER BY rh.Date
'''

query


"SELECT SecurityId, TReturn, Date, Flag\n--select count(1)\nFROM EquityResearch.dbo.ReturnsHistory rh\n\tINNER JOIN securitydb.dbo.SecurityMaster_Hist sm\n\t\tON rh.IntegerId = sm.IntegerId\n\t\tAND rh.Date BETWEEN sm.StartDate AND sm.EndDate\n\t\t-- AND LEFT(rh.IntegerId,1) = '1' --for US names\nAND rh.Date IN ('2019-01-31', '2019-02-28', '2019-03-29')\nAND sm.Ticker IN ('AMZN', 'AAPL')\nORDER BY rh.Date\n"

In [5]:
# running the query to get the data into a variable in Python
ret = pd.read_sql_query(query,conn)
ret

,SecurityId,TReturn,Date,Flag
0,AMZN,87764.937500,2019-01-31,0
1,AAPL,40222.238281,2019-01-31,0
2,AAPL,42023.039062,2019-02-28,0
3,AMZN,83736.000000,2019-02-28,0
4,AMZN,90931.906250,2019-03-29,0
5,AAPL,46100.359375,2019-03-29,0


In [6]:
# reorder df by Date (REQUIRED for using pct_change function)
ret.sort_values(by = ['SecurityId', 'Date'], inplace=True)

In [7]:
# calculate forward one month return
ret['MthRet'] = ret.groupby(['SecurityId']).TReturn.pct_change().shift(periods=-1)*100

In [8]:
ret

,SecurityId,TReturn,Date,Flag,MthRet
1,AAPL,40222.238281,2019-01-31,0,4.477127
2,AAPL,42023.039062,2019-02-28,0,9.702583
5,AAPL,46100.359375,2019-03-29,0,NaN
0,AMZN,87764.937500,2019-01-31,0,-4.590600
3,AMZN,83736.000000,2019-02-28,0,8.593563
4,AMZN,90931.906250,2019-03-29,0,NaN


### 2. An example function
Sometimes you may work with a large dataset (e.g. with thousands of days) that you don't want to pull ALL the security return data in one try, as the large data will take too much of your machine's memory.

You may want to consider create a function and call the function in your loop for calculating daily factors.

Below is an simple example to create such a function.

In [15]:
# create Timer class for future use
import time
class Timer(object):
    def __init__(self, name=None):
        self.name = name

    def __enter__(self):
        self.tstart = time.time()

    def __exit__(self, type, value, traceback):
        if self.name:
            print('[%s]' % self.name,)
        print('Elapsed: %s' % (time.time() - self.tstart))

In [16]:
def sec_ret_pull(in_sd, in_ed, in_secid, in_c = conn):
    """
    The function takes in: 
        start date,
        end date,
        security id (can be either str or list)
        sql connection
    
    and output a dataframe of below columns:
        Date
        Security Id
        Interger Id
        Ticker
        Cusip
        Sedol
        Open_
        Close_
        Total Return
        MarketCap
        Flag
        ...
    """
    import pandas as pd
    
    # check secid input type and convert it to string
    if type(in_secid) == str:
        # do nothing
        pass
    elif type(in_secid) == list:
        # convert it to a string & strip the brackets
        in_secid = str(in_secid).strip('[]')

    sql_q = f'''SELECT Date, SecurityId, sm.IntegerId, Ticker, Cusip, Sedol, Open_, Close_, TReturn, MarketCap, Flag
    FROM EquityResearch.dbo.ReturnsHistory rh
        INNER JOIN securitydb.dbo.SecurityMaster_Hist sm
            ON rh.IntegerId = sm.IntegerId
            AND rh.Date BETWEEN sm.StartDate AND sm.EndDate
    AND rh.Date BETWEEN '{in_sd}' AND '{in_ed}'
    AND Ticker IN ({in_secid})
    ORDER BY Ticker, rh.Date
    '''
    
    with Timer('SQL Return Data Pull'):
        output = pd.read_sql_query(sql_q,in_c)
    return output

In [17]:
Security = ['AMZN','AAPL'] 

sec_ret_pull('03/01/2019', '03/05/2019', Security)

[SQL Return Data Pull]
Elapsed: 0.9080028533935547


,Date,SecurityId,IntegerId,Ticker,Cusip,Sedol,Open_,Close_,TReturn,MarketCap,Flag
0,2019-03-01,AAPL,16027,AAPL,03783310,204625,174.279999,174.970001,42464.750000,8.250325e+11,0
1,2019-03-04,AAPL,16027,AAPL,03783310,204625,175.690002,175.850006,42678.320312,8.291820e+11,0
2,2019-03-05,AAPL,16027,AAPL,03783310,204625,175.940002,175.529999,42600.660156,8.276731e+11,0
3,2019-03-01,AMZN,13892,AMZN,02313510,200001,1655.130005,1671.729980,85364.937500,8.211588e+11,0
4,2019-03-04,AMZN,13892,AMZN,02313510,200001,1685.000000,1696.170044,86612.937500,8.331638e+11,0
5,2019-03-05,AMZN,13892,AMZN,02313510,200001,1702.949951,1692.430054,86421.960938,8.313267e+11,0
